In [150]:
import os
from ctypes import *

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [151]:
path_to_dll = "C:/Users/Maathess/Desktop/Projet_annuel_flag/PMC/cmake-build-debug/PMC2.dll"
mylib = cdll.LoadLibrary(path_to_dll)

In [262]:
def import_images_and_assign_labels(
        folder, label, X, Y
):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8,8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)


def import_dataset():
    dataset_folder = "C:/Users/Maathess/Desktop/Projet_annuel_flag/Datasets"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    import_images_and_assign_labels(
        os.path.join(train_folder, "brazil_flag"), [1,0,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "french_flag"), [0,1,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "ireland_flag"), [0,0,1], X_train, y_train
    )
    
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "brazil_flag"), [1,0,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "french_flag"), [0,1,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "ireland_flag"), [0,0,1], X_test, y_test
    )

    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))


In [263]:
def run():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    dataset_inputs = np.array(X_train)
    dataset_expected_outputs = np.array(y_train)

    init_tab = [2, 3]
    init_size = len(init_tab)
    init_type = c_int * init_size
    init = init_type(*init_tab)

    mylib.create_mlp_model.argtypes = [init_type, c_int]
    mylib.create_mlp_model.restype = c_void_p

    model = mylib.create_mlp_model(init, int(init_size))
    test_dataset = X_test
    img_test= [X_test[0]]
    colors = ["blue" if np.argmax(output) == 0 else ("red" if np.argmax(output) == 1 else "green") for output in dataset_expected_outputs]

    mylib.getXSize.argtypes = [c_void_p]
    mylib.getXSize.restype = c_int
    tmp_len = mylib.getXSize(model)

    flattened_dataset_inputs = []
    for p in dataset_inputs :
        flattened_dataset_inputs.append(p[0])
        flattened_dataset_inputs.append(p[1])

    arrsize_flat = len(flattened_dataset_inputs)
    arrtype_flat = c_float * arrsize_flat
    arr_flat = arrtype_flat(*flattened_dataset_inputs)

    arrsize_exp = len(flattened_dataset_inputs)
    arrtype_exp = c_float * arrsize_exp
    arr_exp = arrtype_exp(*flattened_dataset_inputs)
    
    predicted_outputs = []
    for p in img_test :
        arrsizeP = len(p)
        arrtypeP = c_float * arrsizeP
        arrP = arrtypeP(*p)
        mylib.predict_mlp_model_classification.argtypes = [c_void_p, arrtypeP]
        mylib.predict_mlp_model_classification.restype = POINTER(c_float)
        tmp = []

        tmp = mylib.predict_mlp_model_classification(model, arrP)
        np_arr = np.ctypeslib.as_array(tmp, (tmp_len,))
        predicted_outputs.append(np_arr)
    
    print(predicted_outputs)
    
    mylib.destroy_MLP(model)

In [264]:
if __name__ == "__main__":
    run()

[array([0.3765885 , 0.25319484, 0.8728913 ], dtype=float32)]


ArgumentError: argument 1: <class 'OverflowError'>: int too long to convert

In [273]:
def run_train():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    dataset_inputs = np.array(X_train)
    dataset_expected_outputs = np.array(y_train)

    init_tab = [2,32,3]
    init_size = len(init_tab)
    init_type = c_int * init_size
    init = init_type(*init_tab)

    mylib.create_mlp_model.argtypes = [init_type, c_int]
    mylib.create_mlp_model.restype = c_void_p

    model1 = mylib.create_mlp_model(init, int(init_size))
    test_dataset = X_test
    img_test = [X_test[0]]
    colors = ["blue" if np.argmax(output) == 0 else ("red" if np.argmax(output) == 1 else "green") for output in dataset_expected_outputs]

    mylib.getXSize.argtypes = [c_void_p]
    mylib.getXSize.restype = c_int
    tmp_len = mylib.getXSize(model1)

    flattened_dataset_inputs = []
    for p in dataset_inputs :
        flattened_dataset_inputs.append(p[0])
        flattened_dataset_inputs.append(p[1])
        
    flattened_dataset_outputs = []
    for p in dataset_expected_outputs:
        flattened_dataset_outputs.append(p[0])
        flattened_dataset_outputs.append(p[1])
        flattened_dataset_outputs.append(p[2])

    # definition de train_classification_stochastic_gradient....
    arrsize_flat = len(flattened_dataset_inputs)
    arrtype_flat = c_float * arrsize_flat
    arr_flat = arrtype_flat(*flattened_dataset_inputs)

    arrsize_exp = len(flattened_dataset_outputs)
    arrtype_exp = c_float * arrsize_exp
    arr_exp = arrtype_exp(*flattened_dataset_outputs)

    mylib.train_classification_stochastic_gradient_backpropagation_mlp_model.argtypes = [c_void_p, arrtype_flat, c_int,
                                                                                         arrtype_exp, c_float, c_int]
    mylib.train_classification_stochastic_gradient_backpropagation_mlp_model.restype = None

    mylib.train_classification_stochastic_gradient_backpropagation_mlp_model(model1, arr_flat, arrsize_flat, arr_exp,
                                                                             0.001, 10000)
    predicted_outputs = []
    for p in img_test :
        arrsizeP = len(p)
        arrtypeP = c_float * arrsizeP
        arrP = arrtypeP(*p)
        mylib.predict_mlp_model_classification.argtypes = [c_void_p, arrtypeP]
        mylib.predict_mlp_model_classification.restype = POINTER(c_float)

        tmp = mylib.predict_mlp_model_classification(model1, arrP)
        np_arr = np.ctypeslib.as_array(tmp, (tmp_len,))
        predicted_outputs.append(np_arr)

    print(predicted_outputs)
    

    mylib.destroy_MLP(model1)

In [274]:
if __name__ == "__main__":
    run_train()

[array([0.37660575, 0.30911642, 0.32637352], dtype=float32)]


ArgumentError: argument 1: <class 'OverflowError'>: int too long to convert